---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [4]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

%matplotlib notebook

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [27]:
def graph_identification():
    
    cluster, shortest_path = [],[]
    
    for count, G in enumerate(P1_Graphs):
        degrees = G.degree()
        degree_vals = sorted(set(degrees.values()))
        #print(list(degrees.values()))
        histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_vals]
        #plt.plot(degree_vals,histogram, label='Graph {}'.format(count))
        
        cluster.append(nx.average_clustering(G))
        shortest_path.append(nx.average_shortest_path_length(G))
    
    #plt.legend()
    #plt.xscale('log')
    #plt.yscale('log')
    #print(cluster)
    #print(shortest_path)
    
    return ['PA','SW_L', 'SW_H', 'PA','PA']

graph_identification()

['PA', 'SW_L', 'SW_H', 'PA', 'PA']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [28]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))
G.nodes(data=True)[:5]

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


[(0, {'Department': 1, 'ManagementSalary': 0.0}),
 (1, {'Department': 1, 'ManagementSalary': nan}),
 (2, {'Department': 21, 'ManagementSalary': nan}),
 (3, {'Department': 21, 'ManagementSalary': 1.0}),
 (4, {'Department': 21, 'ManagementSalary': 1.0})]

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [46]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler

def is_management(node):
    salary = node[1]['ManagementSalary']
    if salary == 1:
        return 1
    elif salary == 0:
        return 0
    else:
        return None

def salary_predictions():
    
    df = pd.DataFrame(index=G.nodes())
    df['is_management'] = pd.Series([is_management(node) for node in G.nodes(data=True)])
    df['degree'] = pd.Series(G.degree())
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G,normalized=True))
    df['betweenness'] = pd.Series(nx.betweenness_centrality(G,normalized=True))
    
    df_train = df[~pd.isnull(df['is_management'])]
    df_test = df[pd.isnull(df['is_management'])]
    
    scaler = MinMaxScaler()
    features = ['degree','degree_centrality','clustering','closeness','betweenness']
    X_train = scaler.fit_transform(df_train[features])
    X_test = scaler.transform(df_test[features])
    y_train = df_train['is_management']

    clf = MLPClassifier(hidden_layer_sizes = [10,10], alpha=1, random_state=0, solver='lbfgs')
    clf.fit(X_train,y_train)
    predictions = clf.predict_proba(X_test)[:,1]
    
    return pd.Series(predictions,df_test.index)

salary_predictions()

1       0.014901
2       0.804624
5       1.000000
8       0.148151
14      0.049702
18      0.239154
27      0.344935
30      0.402536
31      0.107530
34      0.097065
37      0.016749
40      0.050634
45      0.152294
54      0.536377
55      0.869577
60      0.047872
62      0.999972
65      0.965366
77      0.028918
79      0.063410
97      0.033014
101     0.008802
103     0.248434
108     0.053881
113     0.336682
122     0.006274
141     0.358306
142     0.670192
144     0.022479
145     0.777511
          ...   
913     0.006094
914     0.011653
915     0.000872
918     0.127744
923     0.003272
926     0.022613
931     0.006952
934     0.002942
939     0.002278
944     0.000971
945     0.004142
947     0.144533
950     0.012196
951     0.004167
953     0.004828
959     0.001269
962     0.000945
963     0.135166
968     0.019715
969     0.021642
974     0.038897
984     0.002696
987     0.052464
989     0.024786
991     0.035269
992     0.001657
994     0.001356
996     0.0010

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [47]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [64]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler

def new_connections_predictions():
    
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']
    
    node1, node2, preferentials = zip(*list(nx.preferential_attachment(G, future_connections.index)))
    future_connections['preferential_attachment'] = preferentials
    node1, node2, csh = zip(*list(nx.cn_soundarajan_hopcroft(G, future_connections.index)))
    future_connections['cn_soundarajan_hopcroft'] = csh
    node1, node1, res_alloc = zip(*list(nx.resource_allocation_index(G, future_connections.index)))
    future_connections['resource_allocation'] = res_alloc
    node1, node2, jaccard = zip(*list(nx.jaccard_coefficient(G, future_connections.index)))
    future_connections['jaccard_coefficient'] = jaccard
    
    scaler = MinMaxScaler()
    df_train = future_connections[~pd.isnull(future_connections['Future Connection'])]
    df_test = future_connections[pd.isnull(future_connections['Future Connection'])]
    features = ['preferential_attachment','cn_soundarajan_hopcroft','resource_allocation','jaccard_coefficient']
    X_train = scaler.fit_transform(df_train[features])
    X_test = scaler.transform(df_test[features])
    y_train = df_train['Future Connection']
    
    clf = MLPClassifier(hidden_layer_sizes = [10,10], alpha=1, random_state=0, solver='lbfgs')
    clf.fit(X_train,y_train)
    predictions = clf.predict_proba(X_test)[:,1]
    
    return pd.Series(predictions,df_test.index)

new_connections_predictions()

(107, 348)    0.029439
(542, 751)    0.011120
(20, 426)     0.612765
(50, 989)     0.011259
(942, 986)    0.011409
(324, 857)    0.011239
(13, 710)     0.156446
(19, 271)     0.114755
(319, 878)    0.011307
(659, 707)    0.011134
(49, 843)     0.011351
(208, 893)    0.011167
(377, 469)    0.006482
(405, 999)    0.020059
(129, 740)    0.017420
(292, 618)    0.020735
(239, 689)    0.011317
(359, 373)    0.008169
(53, 523)     0.092472
(276, 984)    0.011343
(202, 997)    0.011372
(604, 619)    0.065551
(270, 911)    0.011314
(261, 481)    0.071761
(200, 450)    0.995799
(213, 634)    0.011077
(644, 735)    0.053074
(346, 553)    0.010736
(521, 738)    0.010098
(422, 953)    0.018368
                ...   
(672, 848)    0.011314
(28, 127)     0.959934
(202, 661)    0.010679
(54, 195)     0.999950
(295, 864)    0.011218
(814, 936)    0.011098
(839, 874)    0.011409
(139, 843)    0.011230
(461, 544)    0.009666
(68, 487)     0.009592
(622, 932)    0.011184
(504, 936)    0.016238
(479, 528) 